In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import random
from kuramoto import Kuramoto

In [2]:
def sample_NWS(num_samples, nodes, neighbors, probability, half_sync=False):
    df = pd.DataFrame(columns=['kappa', '# Edges', '# Nodes', 'Min Degree', 'Max Degree', 'Diameter', 'Concentrated'])
    
    for i in range(num_samples):
        
        G = nx.newman_watts_strogatz_graph(nodes, neighbors, probability)
        kappa = random.uniform(0.5, 4.5)
        
        if nx.is_connected(G):
            # Number of Edges and Nodes
            edges = G.number_of_edges()
            nodes = G.number_of_nodes()

            # Min and Max degree
            degree_sequence = sorted((d for n, d in G.degree()), reverse=True)
            dmax = max(degree_sequence)
            dmin = min(degree_sequence)

            # Diameter of graph
            diam = nx.diameter(G)
            
            # Applying Kuramoto
            adj_mat = nx.to_numpy_array(G)
            
            if half_sync:
                natfreqs = np.repeat(2 * np.pi * 0, nodes)
                model = Kuramoto(coupling=kappa, dt=0.01, T=50, natfreqs=natfreqs, half_sync=half_sync)
            else:
                model = Kuramoto(coupling=kappa, dt=0.01, T=50, n_nodes=nodes, half_sync=half_sync)
            
            sim = model.run(adj_mat)
            conc = int(model.concentrated)

            df.at[len(df.index)] = [kappa, edges, nodes, dmin, dmax, diam, conc]
    
    return df

In [3]:
num_samples = 10000
nodes = 50
neighbors = 4
probability = 0.80
half_sync = False

df_NWS = sample_NWS(num_samples, nodes, neighbors, probability)
df_NWS

,kappa,# Edges,# Nodes,Min Degree,Max Degree,Diameter,Concentrated
0,3.427979,182,50,4,11,4,0
1,4.279666,180,50,5,11,4,1
2,0.76703,180,50,5,10,4,0
3,1.288323,179,50,5,10,4,0
4,3.286695,187,50,5,11,4,0
...,...,...,...,...,...,...,...
9995,1.027659,182,50,4,12,4,0
9996,3.613917,178,50,4,11,4,1
9997,3.076541,169,50,4,10,4,0
9998,1.775517,174,50,4,9,4,0


In [4]:
sum(df_NWS['Concentrated'])

1838

In [5]:
path = f'./datasets/newman_{nodes}_{neighbors}_{probability}_{num_samples}_{half_sync}'
df_NWS.to_csv(path)

In [6]:
def sample_ER(num_samples, half_sync=False):
    df = pd.DataFrame(columns=['kappa', '# Edges', '# Nodes', 'Min Degree', 'Max Degree', 'Diameter', 'Concentrated'])
    
    for i in range(num_samples):
        
        n = random.randint(25, 100)
        p = random.random()
        kappa=random.uniform(0.5, 4.5)
        G = nx.erdos_renyi_graph(n, p)
        
        if nx.is_connected(G):
            # Number of Edges and Nodes
            edges = G.number_of_edges()
            nodes = G.number_of_nodes()

            # Min and Max degree
            degree_sequence = sorted((d for n, d in G.degree()), reverse=True)
            dmax = max(degree_sequence)
            dmin = min(degree_sequence)

            # Diameter of graph
            diam = nx.diameter(G)
            
            # Applying Kuramoto
            adj_mat = nx.to_numpy_array(G)
            
            if half_sync:
                natfreqs = np.repeat(2 * np.pi * 0, nodes)
                model = Kuramoto(coupling=kappa, dt=0.01, T=50, natfreqs=natfreqs, half_sync=half_sync)
            else:
                model = Kuramoto(coupling=kappa, dt=0.01, T=50, n_nodes=nodes, half_sync=half_sync)
                
            sim = model.run(adj_mat)
            conc = int(model.concentrated)

            # Create dataframe
            df.at[len(df.index)] = [kappa, edges, nodes, dmin, dmax, diam, conc]
    
    return df

In [7]:
num_samples = 10000
half_sync = False

df_ER = sample_ER(num_samples)
df_ER

,kappa,# Edges,# Nodes,Min Degree,Max Degree,Diameter,Concentrated
0,3.887704,3493,89,70,85,2,0
1,3.256588,517,38,22,32,2,1
2,3.712963,3127,92,58,77,2,0
3,1.491359,2489,88,46,69,2,0
4,1.685228,2189,73,53,67,2,0
...,...,...,...,...,...,...,...
9143,2.545986,966,50,31,46,2,0
9144,1.887804,515,42,19,31,2,0
9145,1.478511,2033,84,35,58,2,0
9146,4.484689,576,89,7,21,3,0


In [8]:
sum(df_ER['Concentrated'])

2334

In [9]:
path = f'./datasets/erdosrenyi_{half_sync}'
df_ER.to_csv(path)

In [42]:
def gen_KM(r, num_samples, half_sync=False):
    """
    
    """
    df = pd.DataFrame() # Master df
    
    for i in range(num_samples):
        
        df1 = pd.DataFrame(columns=['# Edges', '# Nodes', 'Min Degree', 'Max Degree', 'Diameter', 'Concentrated'])
        temp = pd.DataFrame() # Dynamics df
        
        nodes = 500
        neighbors = int(random.uniform(1, 25))
        probability = random.uniform(0, 1)
        G = nx.newman_watts_strogatz_graph(nodes, neighbors, probability)
        K = 1.96
        
        if nx.is_connected(G):
            # Number of Edges and Nodes
            edges = G.number_of_edges()
            nodes = G.number_of_nodes()

            # Min and Max degree
            degree_sequence = sorted((d for n, d in G.degree()), reverse=True)
            dmax = max(degree_sequence)
            dmin = min(degree_sequence)

            # Diameter of graph
            diam = nx.diameter(G)

            # Applying Kuramoto
            adj_mat = nx.to_numpy_array(G)

            if half_sync:
                natfreqs = np.repeat(2 * np.pi * 0, nodes)
                model = Kuramoto(coupling=K, dt=0.01, T=25, natfreqs=natfreqs, half_sync=half_sync)
            else:
                model = Kuramoto(coupling=K, dt=0.01, T=25, n_nodes=nodes, half_sync=half_sync)

            sim = model.run(adj_mat)
            conc = int(model.concentrated)
            
            df1.at[len(df1.index)] = [edges, nodes, dmin, dmax, diam, conc]

            print(sim[:r,:].flatten(order='C').shape)
            df2 = pd.DataFrame(sim[:r,:].flatten(order='C')).T
            temp = pd.concat([df2, df1], axis=1)

            df = df.append(temp, ignore_index=True)
    
    return temp

In [43]:
gen_KM(250, 1, half_sync=False)

(625000,)


/var/folders/9w/kr1grtwx7gxc9ysp_clrx0y00000gn/T/ipykernel_14144/2321418788.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(temp, ignore_index=True)


,0,1,2,3,4,5,6,7,8,9,...,624996,624997,624998,624999,# Edges,# Nodes,Min Degree,Max Degree,Diameter,Concentrated
0,5.183397,5.181145,5.178904,5.176674,5.174455,5.172247,5.170047,5.167857,5.165675,5.163502,...,2.925967,2.930019,2.934058,2.938084,5170,500,20,26,5,0


In [17]:
int(random.uniform(1, 25))

6